In [1]:
import csv
from datetime import datetime, timedelta
import time
import numpy as np
from collections import defaultdict, namedtuple
import sys

Na prática, saber se é caso pai ou filho não interessa muito, pois não vou ver nesse nível

# Setting

In [2]:
pop_size = 100
perc_melhores = 0.3
perc_piores = 0.1
perc_active_mutation = 0.1 #população que sofre mutação
perc_mutacao = 0.05  #percentual dos genes
current_time = datetime(2018,5,7,8,0,0)
minutos_plano = 120

# Importando dados e  criando variáveis auxiliares

In [3]:
demanda = {}
formato = '%Y-%m-%d %H:%M:%S'
with open('demanda.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    
    _ = next(reader)
    
    for row in reader:
        numerocasofilho = '{}_{}'.format(row[0], row[2])
        skill = 'etapa ' + str(row[2]) if row[2] != '1' else row[1]
        #não interessa a data de entrada
        data_final = datetime.strptime(row[4],formato)
        demanda[numerocasofilho] = [skill,data_final]
        
# criando variáveis auxiliares
demanda_etapa =  {}
demanda_prazo =  {}
for key,value in demanda.items():
    demanda_etapa[key] = value[0]
    demanda_prazo[key] = value[1]

# Definindo funcionais para trabalho
A classe grupo simula a forma de trabalho do **salesforce**

In [4]:
class grupo(object):
    
    def __init__(self, current_time):
        self.__prioridades = defaultdict(list)
        self.__cdatetime = current_time
        self.__ocupada = {}
    
    def funcional(self,funcional, skills_tma):
        
        #para controlar para cada funcional se está ocupada
        self.__ocupada[ funcional]= 0 #guarda o minuto em que volta a estar disponível  
        
        for prioridade, skill in enumerate(skills_tma.keys()):
            #pela ordem cada skill vai para uma prioridade (proeficiência)
            self.__prioridades[prioridade].append({skill: [funcional, skills_tma[skill]]})
                
    def atendimento(self, minutos, demanda_etapa, demanda_prazo, individual):
        '''Recebe o minuto de planejamento e indivíduo gerado (demandas ordenadas)'''
        
        #mandar dentro de cada etapa as rds em ordem de atendimento acelera as próximas etapas
        etapa_rd = defaultdict(list)
        _ = {etapa_rd[demanda_etapa[rd]].append(rd) for rd in individual}
        
        #for key in etapa_rd:
            #print('Quantidade{}--{}'.format(key, len(etapa_rd[key])))
        
        sla_rd = 0
        
        #1. A simulação deve consumir as demandas dentro de uma janela de tempo
        for minuto in range(1,minutos+1):
            #print('minuto {}'.format(minuto))
            
            #2. Por prioridade (proeficiência) as demandas devem ser alocadas
            for proeficiencia, skills in self.__prioridades.items():
                
                #3. Acessando os skills da proeficiência
                for skill in skills:
                    
                    #4. Verificar se existe demanda para o skill acessado
                    skill_acessado = list(skill.keys())[0]
                    if skill_acessado in etapa_rd.keys():
                        
                        #5. Verificar se funcional está disponível
                        funcional, tma = skill[skill_acessado][0], skill[skill_acessado][1]
                        
                        if self.__ocupada[funcional] <= minuto:
                            #verificado que existe demanda e a funcional está disponível
                            
                            #retirando seguindo a ordem
                            rd = etapa_rd[skill_acessado].pop(0)
                            
                            
                            #atualizando tempo que a funcional vai ficar ocupada
                            self.__ocupada[funcional] = minuto + tma
                            
                            #Verificando se a demanda foi atendida no prazo
                            #caso positivo podemos criar um novo dado
                            if self.__cdatetime + timedelta(minutes = minuto + tma) <= demanda_prazo[rd]:
                                sla_rd += 1 
                            
                            
                            aux = self.__cdatetime + timedelta(minutes = minuto + tma)
                            '''print('rd: {} minuto: {} hs: {} hr: {} Prazo: {} func: {}'.format(rd,
                                                                                     minuto,
                                                                                     aux,
                                                                                     demanda_prazo[rd],
                                                                                     sla_rd,
                                                                                    funcional))'''
                            
                            
                            
                            #se for último elemento da etapa_rd, então excluo a chave
                            if len(etapa_rd[skill_acessado]) == 0:
                                etapa_rd.pop(skill_acessado)
                        
        #6. Retorno a quantidade de rds feitas no prazo para avaliar a qualidade do indivíduo
        return sla_rd
    
    def restart(self):
        #necessário colocar todas as funcionais como livres
        for funcional in self.__ocupada.keys():
            self.__ocupada[funcional] = 0

# Criando população inicial

In [5]:
def start_pop(demanda,pop_size):
    #ordem original -- não dever ser alterado
    ordem = np.array(list(demanda.keys()), dtype=np.str)
    pop = {}

    for ii in range(pop_size):
        ordem_atendimento = ordem.copy()
        np.random.shuffle(ordem_atendimento)
        pop[ii] = list(ordem_atendimento)

    return pop

# Escolhendo os indivíduos mais promissores
Os indivíduos que possuem os melhores resultados são mantidos.

Os de menor qualidade são sorteados para que alguns sejam mantidos

In [6]:
def selecionando_geradores(resultado):
    ordenar_individuos = sorted(resultado.items(), key = lambda x:x[1], reverse=True)

    #Guardando os resultados para comparativos futuros
    #keep_results = ordenar_individuos[:]

    #melhores indivíduos
    best_individual = [i for i in ordenar_individuos[:int(perc_melhores*pop_size)]]

    #sorteando os piores
    worst_individual = [i[0] for i in ordenar_individuos[int(perc_melhores*pop_size):]]
    rand_worst = list(np.random.choice(worst_individual, replace= False, size= int(perc_piores * len(worst_individual))))

    rand_worst = [i for i in ordenar_individuos if i[0] in rand_worst]

    #indivíduos para nova etapa
    individuals = best_individual + rand_worst

    return individuals

# Breeding

Criar filhos a partir do DNA dos pais

In [7]:
def breeding(individuals, pop):
    #1.Identificar quem vai trocar material entre si
    casais = np.random.choice([i[0] for i in individuals],replace=True,size=[pop_size,2])

    #2.Regra de troca de DNA -- sorteio aleatório
    for i, casal in enumerate(casais):

        #posso usar várias vezes o mesmo individuo para vários cruzamentos
        aux, breed= {}, []

        aux[casal[0]] = pop[casal[0]].copy()
        aux[casal[1]] = pop[casal[1]].copy()

        while len(aux[casal[0]]) and len(aux[casal[1]]):
            choice = np.random.choice([casal[0], casal[1]])

            #sempre descarto
            rd = aux[choice].pop(0)

            #se ainda não existe no gene do filho eu add, caso contrário descarto
            if not rd in breed:
                breed.append(rd)

        #crio a nova geração sobre a antiga
        pop[i] = breed[:]
        
    return pop

# Mutation
A mutação acontece adicionando novos indivíduos a população. Não mudos os que já existem, simplesmente sorteio alguns e crio um filho com mutação. No caso atual mutação significa trocar algumas posições na ordem das demandas.

In [8]:
def mutation(pop_size, perc_active_mutation, perc_mutacao):
    sample = np.random.choice(range(pop_size), replace=False, size = int(pop_size*perc_active_mutation))
    for choice in sample:

        length = len(pop[choice])
        changes = np.random.choice(range(length), replace = False, size = [int(length*perc_mutacao), 2])

        for change in changes:
            #invertendo os genes
            aux_1 = pop[choice][change[0]][:]
            aux_2 = pop[choice][change[1]][:]

            pop[choice][change[0]] = aux_2
            pop[choice][change[1]] = aux_1
            
    return pop

# Create group and Fit Function

A função para avaliar a qualidade dos indivíduos será a contagem de SLA perdida devido a ordem de seleção das demandas.

Toda demanda selecionada tem as seguintes características:
1. Tem de ser alocada em alguma funcional
2. A funcional precisa ficar um tempo com essa demanda (tempo médio de atendimento)

O resultado é influenciado pela ordem com que escolho as demandas. Cada indivíduo será um vetor que guarda para cada demanda a ordem em que cada demanda foi selecionada.

In [9]:
func_skills = {}
func_skills['0001'] = {'Poderes 1':10, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0002'] = {'Poderes 1':10, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0003'] = {'Poderes 1':10, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0004'] = {'Poderes 1':10, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0005'] = {'Poderes 1':10, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0006'] = {'Poderes 1':10, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0007'] = {'Poderes 1':10, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0008'] = {'Poderes 1':10, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0009'] = {'Poderes 1':10, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0010'] = {'Poderes 1':10, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0011'] = {'Poderes 2':15, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0012'] = {'Poderes 2':15, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0013'] = {'Poderes 2':15, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0014'] = {'Poderes 2':15, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0015'] = {'Poderes 2':15, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0016'] = {'Poderes 2':15, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0017'] = {'Poderes 2':15, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0018'] = {'Poderes 2':15, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0019'] = {'Poderes 2':15, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0020'] = {'Poderes 2':15, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0021'] = {'Poderes 3':20, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0022'] = {'Poderes 3':20, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0023'] = {'Poderes 3':20, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0024'] = {'Poderes 3':20, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0025'] = {'Poderes 3':20, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0026'] = {'Poderes 3':20, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0027'] = {'Poderes 3':20, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0028'] = {'Poderes 3':20, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0029'] = {'Poderes 3':20, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0030'] = {'Poderes 3':20, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0031'] = {'Poderes 4':25, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0032'] = {'Poderes 4':25, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0033'] = {'Poderes 4':25, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0034'] = {'Poderes 4':25, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0035'] = {'Poderes 4':25, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0036'] = {'Poderes 4':25, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0037'] = {'Poderes 4':25, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0038'] = {'Poderes 4':25, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0039'] = {'Poderes 4':25, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0040'] = {'Poderes 4':25, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0041'] = {'Poderes 5':30, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0042'] = {'Poderes 5':30, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0043'] = {'Poderes 5':30, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0045'] = {'Poderes 5':30, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0044'] = {'Poderes 5':30, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0046'] = {'Poderes 5':30, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0047'] = {'Poderes 5':30, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0048'] = {'Poderes 5':30, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0049'] = {'Poderes 5':30, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0050'] = {'Poderes 5':30, 'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0051'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0052'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0053'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0054'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0055'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0056'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0057'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0058'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0059'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0060'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0061'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0062'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0063'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0064'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0065'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0066'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0067'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0068'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0069'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}
func_skills['0070'] = {'etapa 2': 5, 'etapa 3': 5, 'etapa 4': 5, 'etapa 5': 5}

In [10]:
g = grupo(current_time=current_time)
for func, skill_tma in func_skills.items():
    g.funcional(funcional=func, skills_tma=skill_tma)

In [11]:
pop = start_pop(demanda=demanda, pop_size=pop_size)

best_result_list = []
best_result = 0
for epochs in range(200):
    start = time.time()
    
    resultado = {}
    for ii in range(pop_size):

        resultado[ii] = g.atendimento(minutos_plano, demanda_etapa.copy(),demanda_prazo.copy(), pop[ii].copy())
        g.restart()

    #selecionar geradores
    individuals = selecionando_geradores(resultado=resultado)

    #breeding
    pop = breeding(individuals, pop)

    #mutation
    pop = mutation(pop_size, perc_active_mutation, perc_mutacao)
    
    end = time.time() - start
    
    #best result
    aux = sorted(resultado.items(), key=lambda x:x[1], reverse=True)[0]
    if aux[1] > best_result:
        best_result = aux[1]
        best_result_list = pop[aux[0]]
    
    sys.stdout.write('\rEpoch {} - Melhor {} - Pior {} Time {}'.format(epochs, 
                                                                       max(resultado.values()),
                                                                       min(resultado.values()),
                                                                       end))

Epoch 199 - Melhor 474 - Pior 469 Time 2.6791791915893555